## Imports

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

import torch
import torch.nn as nn
import pandas as pd

from collections import OrderedDict 
from sklearn import metrics, model_selection
from torch.optim import Adam

sys.path.append(os.path.abspath(''))

import utils.more_torch_functions as mtf
import datasets

from utils.custom_activations import StepActivation, StepFunction
from utils.modules import Parallel, MaxLayer, MaxHierarchicalLayer
from utils.misc import train_model, cov_score

# torch.autograd.set_detect_anomaly(True)

## Load data

In [2]:
np_x, np_y = datasets.DiabetesDataset.get_dataset(balancing=True, discretizing=False, hot_encoding=True)
x_data, y_data = torch.Tensor(np_x), torch.Tensor(np_y)
input_size = x_data.size(1)
print(x_data.size())

torch.Size([536, 8])


## Hooks

In [3]:
intermediate_outputs = {}
def get_intermediate_outputs(name):
    def hook(model, input, output):
        if model.training:
            intermediate_outputs.setdefault(name, dict())["train"] = output
        else:
            intermediate_outputs.setdefault(name, dict())["valid"] = output
    return hook

def true_label_for_backward(train, valid):
    def hook(model, input):
        if model.training:
            model.true_labels = train
        else:
            model.true_labels = valid
    return hook

# créer hook fonction de perte pour meilleur backward ? (comparer individuellement les sorties des réseaux ???)

## Networks

### Network parts

In [4]:
class ApproxNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        hl1 = 10

        self.nn = nn.Sequential(OrderedDict([
            ('l1', nn.Linear(input_size,hl1)),
            ('a1', StepActivation()),
            ('l2', nn.Linear(hl1,1)),
            ('a2', StepActivation())
        ]))        

    def forward(self, x):
        x = self.nn(x)

        return x

class BigNet(nn.Module):
    def __init__(self):
        super().__init__()

        hl1 = 50
        hl2 = 25

        self. nn = nn.Sequential(OrderedDict([
            ('l1', nn.Linear(input_size,hl1)),
            ('a1', nn.Sigmoid()),
            ('l2', nn.Linear(hl1,hl2)),
            ('a2', nn.Sigmoid()),
            ('l3', nn.Linear(hl2,1)),
            ('a3', StepActivation()),
        ]))
    
    def forward(self, x):
        x = self.nn(x)

        return x

### Network definition

In [5]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        self.net = nn.Sequential(OrderedDict([
            ('nets', Parallel(OrderedDict([
                ('apx1', ApproxNet()),
                ('apx2', ApproxNet()),
                ('apx3', ApproxNet()),
            ]))),
            ('or_', MaxLayer()),
        ]))

    def forward(self, input):
        return self.net(input)

## Network evaluation

In [6]:
model = Net()
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=1e-2, weight_decay=1e-6)

model.net.nets.register_forward_hook(get_intermediate_outputs("parallel_out"))
train_index, valid_index = torch.utils.data.random_split(range(x_data.size(0)), [0.9, 0.1])

x_train, y_train = x_data[train_index], y_data[train_index]
x_valid, y_valid = x_data[valid_index], y_data[valid_index]

train_model(x_train, y_train, model, criterion, optimizer, 500)
model.eval()
pred_train = model(x_train).detach()
pred_valid = model(x_valid).detach()

print(metrics.f1_score(y_train, pred_train), cov_score(y_train, pred_train))
print(metrics.f1_score(y_valid, pred_valid), cov_score(y_valid, pred_valid))

0.851063829787234 {0.0: 0.768595041322314, 1.0: 0.9128630705394191}
0.8387096774193549 {0.0: 0.6538461538461539, 1.0: 0.9629629629629629}


In [7]:
for param in model.parameters():
    param.requires_grad = False

model.net.nets.add_module("nn", BigNet())

In [8]:
train_model(x_train, y_train, model, criterion, optimizer, 500)
model.eval()
pred_train = model(x_train).detach()
pred_valid = model(x_valid).detach()

print(metrics.f1_score(y_train, pred_train), cov_score(y_train, pred_train))
print(metrics.f1_score(y_valid, pred_valid), cov_score(y_valid, pred_valid))

0.8527131782945737 {0.0: 0.7727272727272727, 1.0: 0.9128630705394191}
0.8387096774193549 {0.0: 0.6538461538461539, 1.0: 0.9629629629629629}


### Comments

very small increase in both metrics (sometimes even decrease in performances)